In [ ]:

import warnings
warnings.filterwarnings('ignore')
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import pymssql
import datetime
import time
import plotly.offline as pyoff
import plotly.graph_objects as go
import numpy as np
import math
from statsmodels.tsa.seasonal import STL
from scipy.signal import savgol_filter
from lenspy import DynamicPlot

# library

In [ ]:

class sandbox():
    def __init__(self,name,start,end,raw_data,filter_data):
        self.raw_data=raw_data
        self.start=start
        self.end=end
        self.name=name
        self.filter_data=filter_data
    def data_check(self,start,end,name):
        print(start,self.start)
        if self.start==start and self.end==end and name==self.name:
            return True 


def request(device_post_api,utc_time_start,utc_time_end):
    print('requests start')
    data = requests.get('{0}'.format(device_post_api) + "&start={0}&end={1}".format(utc_time_start, utc_time_end))
    print('data get!')
    data_json = json.dumps(data.json())
    info = json.loads(data_json)
    
    info_val = info["feeds"]
    info_lat_lon = info["channel"]
    df = json_normalize(info_val)    
    df = df.rename({'created_at': 'time'}, axis=1)
    return df


def getting_data(device_post_api,utc_time_start,utc_time_end,device_name,rain_path):    
    df_rain = pd.read_csv(rain_path)    
    df_rain['時間']=pd.to_datetime(df_rain['時間'])   
    df_rain = df_rain.set_index("時間", drop=True)
    global sb
    try:
        if sb:
            if sb.data_check(utc_time_start,utc_time_end,device_name) is True:
                print('data is exist!')
                df=sb.raw_data
                filename=device_name+'_download_data.xlsx'
                sb.raw_data.to_excel(filename)
                return df,df_rain
            else:
                raise ValueError()
        else:
            raise ValueError()
    except:        
        try:    
            datetime.datetime.strptime(utc_time_start, "%Y-%m-%d %H:%M")
            datetime.datetime.strptime(utc_time_end, "%Y-%m-%d %H:%M")    
        except:
            print('type error')
        print(device_name)
        df=request(device_post_api,utc_time_start,utc_time_end)
        time_node=df['time'][0]
        print('time_node is ',time_node )
        condition=datetime.datetime.strptime(time_node, "%Y-%m-%dT%H:%M:%SZ")>datetime.datetime.strptime(utc_time_start, "%Y-%m-%d %H:%M")
        check_point=df['time'][0]
        print('check_point is ',check_point )
        while condition:
            df1=request(device_post_api,utc_time_start,time_node)
            time_node=df1['time'][0]
            if time_node==check_point:
                break
            else:
                df=df.append(df1)
                check_point=time_node
        df=df.sort_values(by=['time'])
        df.reset_index(inplace=True,drop=True)

        df["time"]=pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%SZ")
        df["time"]=df["time"]+ datetime.timedelta(hours=8)

#         for i in tqdm(range(len(df))):
#             #df["time"][i] = datetime.datetime.strptime(df["time"][i], "%Y-%m-%dT%H:%M:%SZ")
#             df["time"][i] = (df["time"][i] + datetime.timedelta(hours=8))#.strftime("%Y-%m-%d %H:%M:%S")


        print('getting_data is compelete!!')
        sb=sandbox(device_name,utc_time_start,utc_time_end,df,0)
        filename=device_name+'_download_data.xlsx'
        sb.raw_data.to_excel(filename)
        return sb.raw_data,df_rain

# def filter_by_differ(data1,value,times,interpolate=True):
#     data1['waterlevel']=data1['waterlevel'].astype(float)
#     for i in range(times):
#         data1=data1.join(data1.diff().fillna(0),rsuffix='_differ')
        
#         # print(data1)
#         # print(data1)
#         data1=data1.drop(data1[data1['waterlevel_differ']>value].index,axis=0)
#         data1=data1.drop(data1[data1['waterlevel_differ']<(value*-1)].index,axis=0)
#         data1=data1.drop(['waterlevel_differ'],axis=1)        
#         # print(data1.waterlevel.idxmin(),data1.waterlevel.min())
#     # print('done')
#     # print(data1) 
#     if interpolate==True:       
#         data1=data1.resample('10T').mean().interpolate()
#     else:
#         data1=data1.resample('10T').mean()

#     return data1

def filter_by_differ(data1,value,times,interpolate=True):
    col=data1.columns    
    data=pd.Series(data1[col[0]])    
    for i in range(times):
        data_diff=data.diff().fillna(0)
        # print(data)
        # print(data.index)
        # print(data_diff)
        # print(data_diff.index)
        # data1=data1.join(data1.diff().fillna(0),rsuffix='_differ')
        
        # print(data1)
        # print(data1)
        # data=data.drop(data[data_diff>value].index,axis=0)
        data=data.drop(data[data_diff<(value*-1)].index,axis=0)
        # print((data_diff<(value*-1)).index)
        # print(data_diff[data_diff<(value*-1)])
        # data1=data1.drop(data1[data1['waterlevel_differ']<(value*-1)].index,axis=0)
        # data1=data1.drop(['waterlevel_differ'],axis=1)        
        # print(data1.waterlevel.idxmin(),data1.waterlevel.min())
    # print('done')
    # print(data1) 
    # data_after=pd.DataFrame({col[0]:data})
    # print(data_after)
    for i in range(times):
        data_diff=data.diff().fillna(0)
        # print(data)
        # print(data.index)
        # print(data_diff)
        # print(data_diff.index)
        # data1=data1.join(data1.diff().fillna(0),rsuffix='_differ')
        
        # print(data1)
        # print(data1)
        # data=data.drop(data[data_diff>value].index,axis=0)
        data=data.drop(data[data_diff>value].index,axis=0)
        # print((data_diff<(value*-1)).index)
        # print(data_diff[data_diff<(value*-1)])
        # data1=data1.drop(data1[data1['waterlevel_differ']<(value*-1)].index,axis=0)
        # data1=data1.drop(['waterlevel_differ'],axis=1)        
        # print(data1.waterlevel.idxmin(),data1.waterlevel.min())
    # print('done')
    # print(data1) 
    data_after=pd.DataFrame({col[0]:data})
    if interpolate==True:       
        data_after=data_after.resample('10T').mean().interpolate()
    else:
        data_after=data_after.resample('10T').mean()
    return data_after



def sgmodel(data):
    data_filter=savgol_filter(data,window_length=99,polyorder=3)
    return data_filter

def knn_process(df):
    from fancyimpute import KNN
    #先把資料時間校正回10分鐘一筆，有缺的用移漏值代替
    resample_data=sb.filter_data.resample('10T').mean().interpolate(limit = 1)

    #資料取出來
    data=resample_data


    #knn訓練資料前處理

    kn_value=48 #決定幾個為一組訓練
    raw_len=len(data['waterlevel'].values)  #計算原資料長度
    dim_n=int(raw_len/kn_value)  #計算要切成幾層，然後取整數
    after_len=dim_n*kn_value #計算總資料筆數

    #把時間序列轉換成矩陣
    kn_matrix=data['waterlevel'].values[:after_len].reshape((dim_n,kn_value))

    #KNN訓練
    knn_impute = KNN(k = 5,).fit_transform(kn_matrix)

    #把補遺後資料flatten
    aaa=knn_impute.reshape(1,after_len)
    ww=pd.Series(aaa[0])

    #跟原本資料合併
    knn_data=resample_data.iloc[:after_len]
    ww.index=knn_data.index
    knn_data['filter_data']=ww
    #knn_data.plot()
    knn_data=knn_data[knn_data['filter_data']!=0]

    # #畫圖
    # linear=resample_data['waterlevel'].interpolate()

    # trace_raw = go.Scatter(
    #     x= knn_data.index,
    #     y= knn_data['waterlevel'],
    #     name='raw',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines+markers',
    #     line=dict(color='red', width=1)
    # )


    # trace_field5 = go.Scatter(
    #     x= linear.index,
    #     y= linear,
    #     name='linear',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines',
    #     line=dict(color='blue', width=1)
    # )

    # trace_field6 = go.Scatter(
    #     x= knn_data.index,
    #     y= knn_data['filter_data'],
    #     name='knn_plot',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines',
    #     line=dict(color='orange', width=1)
    # )
    # data = [trace_field5, trace_field6,trace_raw]
    # layout = go.Layout({"template": 'ggplot2',
    #                     "title":{"text": '{0}'.format('device_name'), "x": 0.5}, 
    #                     "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0, 0.8]},
    #                     "yaxis":{"title": {"text": "雨量"}},
                        
    #                 "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
    #                         })

    # fig = go.Figure(data=data, layout=layout)
    # fig.show()
    # knn_data.resample('H').mean()['filter_data'].plot()
        
    

    return knn_data['filter_data'].rename('waterlevel')

def filter_by_std(data):
    std=1*(data.rolling(10,min_periods=1,center=True).std())
    mean= data.rolling(10,min_periods=1,center=True).mean()
    upper=(mean+std)
    lower=(mean-std)
    df=pd.DataFrame({'data':data,"upper":upper,'lower':lower})
    condition1=df['data']>df['upper']
    condition2=df['data']<df['lower']
    df=df.drop(df.loc[condition1|condition2].index,axis=0)
    return df['data']

def original_data_process(df,field,high,low,name,point=None,filter_value=None,filter_times=None,knn=False):
    
    time=[]
    value=[]
    for i in range(len(df)):
        if float(df[field][i]) >low and float(df[field][i]) < high and float(df[field][i]) !=0.001:       
            time.append(df['time'][i])
            value.append(float(df[field][i])) 

    data={'time':time,name:value}
    data1 = pd.DataFrame(data)
    data1['time']=pd.to_datetime(data1['time'])
    data1 = data1.set_index("time", drop=True)  
    if filter_value:
        print('before filter point = ',len(data1))
        data1=filter_by_differ(data1,filter_value,filter_times)
        print('after filter point = ',len(data1))
        sb.filter_data=data1
    if knn==True:
        data1=knn_process(data1)
    
    print(data1.median())

    data1=data1.resample('10T').mean().interpolate(limit = point).fillna(data1.median()).round(2)

    data2 = data1.resample('H').mean().round(2)
    

    trace_raw_data = go.Scatter(
    x= data1.index,
    y= data1[name],
    name=name,
    xaxis='x', 
    yaxis='y',
    mode='lines+markers',
    line=dict(color='blue', width=1))
    
    
    trace_resample_data = go.Scatter(
    x= data2.index,
    y= data2[name],
    name=str(name)+'resample',
    xaxis='x', 
    yaxis='y',
    mode='lines+markers',
    line=dict(color='red', width=1))
    data = [trace_raw_data]
    layout = go.Layout({"template": 'ggplot2',
                        "title":{"text": '{0}'.format(name), "x": 0.5}, 
                        "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0, 0.8]},
                        "yaxis":{"title": {"text": name}},
                        "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1}})

    fig = go.Figure(data=data, layout=layout)
    # fig = DynamicPlot(fig)
    fig.show()
    
    return data1,data2


def STL_ana(df,period,factor):
    df_res = df
    print('STL start')
    res = STL(df_res, period).fit()
    fig = res.plot()
    print('STL finish')
    df_res['trend'] = res.trend
    df_res['seasonal'] = res.seasonal
    df_res['residual'] = res.resid
    df_res = df_res.fillna(method='ffill')
    df_res = df_res.fillna(method='bfill')
    df_res['trend']=df_res['trend']+(df_res['residual']*factor)
    return df_res['trend']



def Q_calculate(df_waterlevel,width,slope,man,mode,rolling_number):
    if mode=='1':
        sr_waterlevel = df_waterlevel["waterlevel"]
        A_sr_waterlevel = sr_waterlevel * float(width)
        R_sr_waterlevel = A_sr_waterlevel/((sr_waterlevel*2)+float(width))
        Q_sr_waterlevel = (1/float(man)) * (A_sr_waterlevel) * (R_sr_waterlevel**(2/3)) * ((math.tan(float(slope)/57.3))**(1/2))
        #Q_sr_waterlevel = Q_sr_waterlevel.resample('H').mean().interpolate().rolling(5,min_periods=1,center=True).mean()
        Q_sr_waterlevel = round(Q_sr_waterlevel, 2)
    if mode == '2':
        sr_waterlevel = df_waterlevel["waterlevel"]
        Q_sr_waterlevel =  float(width)*((9.81*(sr_waterlevel**3))**0.5) 
        #Q_sr_waterlevel = Q_sr_waterlevel.resample('H').mean().interpolate().rolling(5,min_periods=1,center=True).mean()
        Q_sr_waterlevel = round(Q_sr_waterlevel, 2)
    return Q_sr_waterlevel




def Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,rolling_number):
    sr_turbidty = pd.Series.ewm(df_turbidty['turbidty'], span=5).mean()   
    sr_turbidty =  (Qs_a * np.log(sr_turbidty)) + Qs_b    
    sr_turbidty=sr_turbidty[sr_turbidty>0]            
    #sr_turbidty = sr_turbidty.resample('H').mean().interpolate().rolling(10,min_periods=1,center=True).mean()  
    Qs_sr_waterlevel = sr_turbidty * 1.1911*(10**(-3)) * Q_sr_waterlevel
    Qs_sr_waterlevel = round(Qs_sr_waterlevel, 2)
    return Qs_sr_waterlevel,sr_turbidty

def draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,
         df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end):
    #======================畫圖區====================================#   
    # trace_field5 = go.Scatter(
    #     x= sr_field5.index,
    #     y= sr_field5['waterlevel'],
    #     name='field5',
    #     xaxis='x', 
    #     yaxis='y2',
    #     mode='lines+markers',
    #     line=dict(color='blue', width=1)
    # )
    
    # trace_field6 = go.Scatter(
    #     x= sr_field6.index,
    #     y= sr_field6['turbidty'],
    #     name='field6',
    #     xaxis='x', 
    #     yaxis='y3',
    #     mode='lines+markers',
    #     line=dict(color='orange', width=1)
    # )
    
    trace1 = go.Scatter(
        x=Q_sr_waterlevel.index,
        y=Q_sr_waterlevel,
        name='流量',
        xaxis='x', 
        yaxis='y2',
        line=dict(color='red', width=1)
    )

    trace2 = go.Bar(
        x=df_rain.index,
        y=df_rain['時雨量'],
        name='rain',
        xaxis='x', 
        yaxis='y1',
        marker=dict(color='rgba(0,0,0,0.3)',
        line=dict(color='rgba(0,0,0,0.3)', width=3))
    )
    
    trace3 = go.Scatter(
        x= df_waterlevel.index,
        y= df_waterlevel['waterlevel'],
        name='水位高',
        xaxis='x', 
        yaxis='y2',
        mode='lines+markers',
        line=dict(color='purple', width=1)
    )
    
    trace4 = go.Scatter(
        x= Qs_sr_waterlevel.index,
        y= Qs_sr_waterlevel,
        name='懸移質輸砂量',
        xaxis='x', 
        yaxis='y3',
        line=dict(color='orange', width=1)
    )
    
    sr_turbidty = go.Scatter(
        x= sr_turbidty.index,
        y= sr_turbidty,
        name='ppm',
        xaxis='x', 
        yaxis='y3',        
        line=dict(color='blue', width=1)
    )
    #===1===#
    # data = [trace2, trace_field5, trace_field6]
    # layout = go.Layout({#"template": 'ggplot2',
    #                     "template": 'plotly_white',
    #                    "title":{"text": '{0}'.format(device_name), "x": 0.5}, 
    #                    "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0, 0.8],
    #                             'range':[utc_time_start, utc_time_end]},
    #                    "yaxis":{"title": {"text": "雨量"}},
    #                    "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right',"title": {"text": "Field5"}},
    #                    "yaxis3": {'anchor': 'free', "overlaying": 'y', "side": 'right',
    #                               'position':0.9,"title": {"text": "Field6"}},
    #                    "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
    #                    #"width": 1000,
    #                    #"height": 500})
    #                     })

    # fig = go.Figure(data=data, layout=layout)
    # fig.show()
    
    #===2===#
    data = [trace2, trace3, sr_turbidty]
    layout = go.Layout({#"template": 'ggplot2',
                        "template": 'plotly_white',
                       "title":{"text": '{0}'.format(device_name), "x": 0.5}, 
                       "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0, 0.8],
                                'range':[utc_time_start, utc_time_end]},
                       "yaxis":{"title": {"text": "雨量"}},
                       "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right',"title": {"text": "Waterlevel"}},
                       "yaxis3": {'anchor': 'free', "overlaying": 'y', "side": 'right',
                                  'position':0.9,"title": {"text": "ppm"}},
                       "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                       #"width": 1000,
                       #"height": 500})
                        })
    # fig = DynamicPlot(fig)
    fig = go.Figure(data=data, layout=layout)

    fig.show()
    
    #===3===#
    data = [trace2, trace1, trace4]
    layout = go.Layout({#"template": 'ggplot2',
                        "template": 'plotly_white',
                       "title":{"text": '{0}'.format(device_name), "x": 0.5}, 
                       "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0,0.8],
                                'range':[utc_time_start, utc_time_end]},
                       "yaxis":{"title": {"text": "雨量"}},
                       "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right',"title": {"text": "Q"}},
                       "yaxis3": {'anchor': 'free', "overlaying": 'y', "side": 'right',
                                  'position':0.9,"title": {"text": "Qs"}},
                       "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                       #"width": 1000,
                       #"height": 500})
                        })
    # fig = DynamicPlot(fig)
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    
def correction_data_save(Q_sr_waterlevel,Qs_sr_waterlevel,waterlevel,sr_turbidty):
    waterlevel=round(waterlevel,2)
    sr_turbidty=round(sr_turbidty,0)
    ppm=round(sr_turbidty*1.1911,0)
    res = pd.DataFrame({'Q(cms)': Q_sr_waterlevel,
                        'Qs(kg/sec)':Qs_sr_waterlevel,
                        'waterlevel(m)':waterlevel,
                        'turbidty(ntu)':sr_turbidty,
                        'turbidty(ppm)':ppm
                        })
    res=res.dropna(subset=['Q(cms)','Qs(kg/sec)'])
    return res
    
def save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,waterlevel):
    Q_sr_waterlevel = Q_sr_waterlevel.resample('H').mean().interpolate().rolling(5,min_periods=1,center=True).mean()
    Qs_sr_waterlevel = Qs_sr_waterlevel.resample('H').mean().interpolate().rolling(5,min_periods=1,center=True).mean()
    waterlevel = waterlevel.resample('H').mean().interpolate().rolling(5,min_periods=1,center=True).mean()
    
    
    waterlevel=round(waterlevel,2)
    Qs_sr_waterlevel=round(Qs_sr_waterlevel,2)
    Q_sr_waterlevel=round(Q_sr_waterlevel,2)
    
    res = pd.DataFrame({'rain(mm/hr)': df_rain['時雨量'], 'Q(cms)': Q_sr_waterlevel, 'Qs(kg/sec)':Qs_sr_waterlevel,
                        'field5':sr_field5['waterlevel'], 'field6':sr_field6['turbidty'],'waterlevel(m)':waterlevel})
    res['turibdty(ppm)']=(res['Qs(kg/sec)']*1000/res['Q(cms)']).round(0)
    res=res.dropna(subset=['field5','field6'])
    res.index.names=['time']
    #cms_peak=res.Q.max()
    #cv_peak=res.Cv.max()
    rain=res['rain(mm/hr)'].sum()
    trans_volume=res['Qs(kg/sec)'].sum()*3600/1000/1.7
    print('===================summary=========================')
    print('累積雨量= '+str(rain)+' (mm)')
    #print(res.tail())  
    #print('洪峰流量= '+str(cms_peak)+' (cms)')
    #print('輸砂濃度= '+str(cv_peak)+' (kg/cms)')
    print('懸移質輸砂量= '+str(int(trans_volume))+' (m3)')
    print('===================summary=========================')
    return res

# 德基_12(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\D2F230武陵20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,4,'waterlevel',filter_value=0.1,filter_times=15)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    #df_waterlevel["waterlevel"] =df_waterlevel["waterlevel"].rolling(6,min_periods=1,center=True).max()
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    #做STL分析---------
    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    
    #----必要-----
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,20)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty'][:'2022-5-1']=df_turbidty['turbidty'][:'2022-5-1']+2
    df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],144,0)       

    #df_turbidty['turbidty']['2021-7-29 14:00':'2021-8-30']=df_turbidty['turbidty']['2021-7-29 14:00':'2021-8-30']+1
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    # df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(20,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,
                  sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'德基_環山三號橋_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'德基_環山三號橋_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371245/feeds.json?',
                       '德基_12', '2022-5-1 00:00', '2022-7-12 14:00', '1', 18, 0.04, 1, 5.49, 1, (-3529), 5138)

# 德基_13(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\D2F230武陵20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,5.3,'waterlevel',point=1,filter_value=0.1,filter_times=5)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,4.4,'turbidty')
 
    #做STL分析---------
    df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"]=df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"].rolling(100,min_periods=1,center=True).max().rolling(100,min_periods=1,center=True).mean()
    df_waterlevel['2021-10-7':'2021-10-10']["waterlevel"]=df_waterlevel['2021-10-7':'2021-10-10']["waterlevel"].rolling(300,min_periods=1,center=True).max().rolling(100,min_periods=1,center=True).mean()

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],48,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,15)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']-0.3
    # df_turbidty['turbidty']['2021-9-29':'2021-12-30']=df_turbidty['turbidty']['2021-9-29':'2021-12-30']+2
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel['waterlevel'])
    res.to_excel(r'德基_南湖溪鋼便橋_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'德基_南湖溪鋼便橋_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371254/feeds.json?',
                       '德基_13南胡溪', '2022-05-01 00:00', '2022-7-30 00:00', '1', 28.5, 0.06, 1.5, 6.24, 1, (-3019), 4633)

# 德基_14(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\D2F230武陵20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',3.6,0,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')

    #做STL分析---------
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel['2021-8-8':'2021-10-10']["waterlevel"]=df_waterlevel['2021-8-8':'2021-10-10']["waterlevel"].rolling(50,min_periods=1,center=True).max().rolling(100,min_periods=1,center=True).mean()

    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)

    #濁度========================================================================================#

    # df_turbidty['turbidty']['2021-8-15':'2021-9-23']=df_turbidty['turbidty']['2021-8-15':'2021-9-23']+3.5
    # df_turbidty['turbidty']['2021-9-14':'2021-9-23']=df_turbidty['turbidty']['2021-9-14':'2021-9-23']+0.5

    #df_turbidty['turbidty']['2021-7-24 09:10:00':'2021-8-03 12:00:00']=abs(0.21-df_turbidty['turbidty']['2021-7-24 09:10:00':'2021-8-03 12:00:00'])
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)

    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel.waterlevel)
    res.to_excel(r'德基_京華橋_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'德基_京華橋_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371256/feeds.json?',
                       '德基_京華橋', '2022-5-1 00:00', '2022-7-12 00:00', '1', 11.5, 0.06, 2, 3.59, 1, (-3239), 5075)

# 霧社_15(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C0L010廬山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,6.8,'waterlevel',filter_times=13,filter_value=0.15)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    # df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"]=df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"].rolling(50,min_periods=1,center=True).max().rolling(100,min_periods=1,center=True).mean()
    # df_waterlevel['2021-10-6':'2021-10-8']["waterlevel"]=7.86
    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],24,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    # df_turbidty['turbidty']['2021-7-10':'2021-7-23']=df_turbidty['turbidty']['2021-7-10':'2021-7-23']+3
    # df_turbidty['turbidty']['2021-7-25':'2021-7-31']=df_turbidty['turbidty']['2021-7-25':'2021-7-31']+3
    # df_turbidty['turbidty']['2021-08-01':'2021-08-04']=df_turbidty['turbidty']['2021-08-01':'2021-08-04']+1.5
    # df_turbidty['turbidty']['2021-08-03':'2021-08-07']=df_turbidty['turbidty']['2021-08-03':'2021-08-07']+3
    # df_turbidty['turbidty']['2021-08-09':'2021-08-11']=df_turbidty['turbidty']['2021-08-09':'2021-08-11']+0.5
    # df_turbidty['turbidty']['2021-08-13':'2021-08-18']=df_turbidty['turbidty']['2021-08-13':'2021-08-18']-0.5
    # df_turbidty['turbidty']['2021-8-20':'2021-9-30']=5-df_turbidty['turbidty']['2021-8-20':'2021-9-30']
    # df_turbidty['turbidty']['2021-10-16':'2021-12-30']=5-df_turbidty['turbidty']['2021-10-16':'2021-12-30']

    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5

    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(500,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,
                  sr_field5,sr_field6,df_waterlevel.waterlevel)
    res.to_excel(r'霧社_榮華巷_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'霧社_榮華巷_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371257/feeds.json?', 
                    '霧社_榮華巷', '2022-05-01 00:00', '2022-7-12 00:00', '1', 55, 0.1, 2, 7.8, 0.95, (-3058), 4664)

# 霧社_16(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C0L010廬山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    # df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"]=df_waterlevel['2021-8-8':'2021-12-30']["waterlevel"].rolling(50,min_periods=1,center=True).max().rolling(100,min_periods=1,center=True).mean()

    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    # df_turbidty['turbidty']=df_turbidty['turbidty']+0.5
    #df_turbidty['turbidty']['2021-7-20 09:10:00':'2021-7-23 12:00:00']=df_turbidty['turbidty']['2021-7-20 09:10:00':'2021-7-23 12:00:00']+0.5
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(12,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5
                  ,sr_field6,df_waterlevel.waterlevel)
    res.to_excel(r'霧社_馬海波橋_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'霧社_馬海波橋_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371258/feeds.json?',
                       '霧社_16馬海波橋', '2022-05-1 00:00', '2022-7-12 00:00', '2', 21.8, 0.06, 3, 4.25, 1.00, (-3208), 4865)

# 霧社_17(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C1H000翠峰20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,5,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    #做STL分析---------
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    #df_waterlevel["waterlevel"]['2021-9-28':'2021-10-11']=df_waterlevel["waterlevel"]['2021-9-28':'2021-10-11'].rolling(40,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(50,min_periods=1,center=True).max()

    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    
    # df_turbidty['turbidty']['2021-7-10':'2021-8-13']=4.5-df_turbidty['turbidty']['2021-7-10':'2021-8-13']
    # df_turbidty['turbidty']['2021-7-10':'2021-8-13']=df_turbidty['turbidty']['2021-7-10':'2021-8-13']-6*(4.2-df_turbidty['turbidty']['2021-7-10':'2021-8-13'])
    # df_turbidty['turbidty']['2021-10-16':'2021-12-30']=df_turbidty['turbidty']['2021-10-16':'2021-12-30']+2
    # df_turbidty['turbidty']['2021-10-6':'2021-10-11']=df_turbidty['turbidty']['2021-10-6':'2021-10-11']+2

    #df_turbidty['turbidty']['2021-7-10':'2021-8-13']=df_turbidty['turbidty']['2021-7-10':'2021-8-13']-0.5

    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],144,0)
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5
                  ,sr_field6,df_waterlevel['waterlevel'])
    res.to_excel(r'霧社_靜翠橋_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'霧社_靜翠橋_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1409019/feeds.json?',
                       '霧社_17靜翠橋', '2022-5-1 00:00', '2022-7-30 00:00', '2', 37.8, 0.06, 6.5, 7.36, 0.95, (-3529), 5138)

# 曾文_18(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,5,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    # df_waterlevel['2021-6-20 05:00:00':'2021-7-15 21:00:00']["waterlevel"]=(df_waterlevel['2021-6-20 05:00:00':'2021-7-15 21:00:00']["waterlevel"]+2.9)
    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+1.5
    df_turbidty['turbidty']['2022-6-6':]=df_turbidty['turbidty']['2022-6-6':]+2
    #df_turbidty['turbidty']['2021-7-20 09:10:00':'2021-7-23 12:00:00']=df_turbidty['turbidty']['2021-7-20 09:10:00':'2021-7-23 12:00:00']+0.5
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel['waterlevel'])
    res.to_excel(r'曾文_達邦_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'曾文_達邦_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1409021/feeds.json?',
                       '曾文_達邦', '2022-05-01 00:00', '2022-7-30 00:00', '1', 60, 0.08, 2, 7.9, 1, (-3109), 4618)

# 曾文_27(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C1M390龍美20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel',filter_times=12,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(100,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]['2021-7-1 05:00:00':'2021-7-15 21:00:00']=df_waterlevel["waterlevel"]['2021-7-1 05:00:00':'2021-7-15 21:00:00']+0.7
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+1.5
    # df_turbidty['turbidty']['2021-7-10':'2021-7-16']=4.5-df_turbidty['turbidty']['2021-7-10':'2021-7-16']
    # df_turbidty['turbidty']['2021-7-10':'2021-7-23']=df_turbidty['turbidty']['2021-7-10':'2021-7-23']+1
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    # df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    # df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],144,0)
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,100)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel['waterlevel'])
    res.to_excel(r'曾文_山美_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'曾文_山美_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1409038/feeds.json?', '曾文_山美',
                       '2022-5-1 00:00', '2022-7-30 00:00', '1', 73, 0.08, 1, 4.9, 1, (-3529), 5138)

# 曾文_28(雨量資料,Qs_a,Qs_b未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C1M390龍美20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]=(df_waterlevel['2021-7-20 05:00:00':'2021-7-21 21:00:00']["waterlevel"]+0.5)*1.05
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(100,min_periods=1,center=True).max()

    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],72,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
    # df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],36,0)       
    df_turbidty['turbidty']=df_turbidty['turbidty']-0.1
    #df_turbidty['turbidty']=df_turbidty['turbidty'].rolling(20,center=True).min()
    #df_turbidty['turbidty']['2021-7-10':'2021-7-16']=df_turbidty['turbidty']['2021-7-10':'2021-7-16']+0.7

    # df_turbidty['turbidty']['2021-7-26':'2021-7-29']=df_turbidty['turbidty']['2021-7-26':'2021-7-29']+0.3
    #df_turbidty['turbidty']['2021-7-20':'2021-7-22']=df_turbidty['turbidty']['2021-7-20':'2021-7-22']+0.7
    # df_turbidty['turbidty']=df_turbidty['turbidty']-0.35
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'曾文_紫荊_0501-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'曾文_紫荊_原始數據_0501-0712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1409040/feeds.json?', '曾文_紫荊',
                       '2022-5-1 00:00', '2022-7-15 00:00', '1', 61, 0.08, 1.5, 5.2, 0.85, (-3033), 4744)

# 隘寮_霧大(雨量資料,所有參數未改)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,r'隘寮.csv')
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2.36,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    df_waterlevel['2021-6-30':'2021-7-22']["waterlevel"]=df_waterlevel['2021-6-30':'2021-7-22']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    df_waterlevel['2021-8-8 15:00':'2021-9-11']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-9-11']["waterlevel"]+0.6
    df_waterlevel['2021-9-18':'2021-10-3']["waterlevel"]=df_waterlevel['2021-9-18':'2021-10-3']["waterlevel"].rolling(30,min_periods=1,center=True).max()
    df_waterlevel['2021-10-3':'2021-10-11']["waterlevel"]=df_waterlevel['2021-10-3':'2021-10-11']["waterlevel"].rolling(50,min_periods=1,center=True).max()

    df_waterlevel['2021-10-20':'2021-10-30']["waterlevel"]=df_waterlevel['2021-10-20':'2021-10-30']["waterlevel"].rolling(40,min_periods=1,center=True).max()

    df_waterlevel['2021-10-12':'2021-10-30']["waterlevel"]=df_waterlevel['2021-10-12':'2021-10-30']["waterlevel"]+0.2

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+2.5
    df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    df_turbidty['turbidty']['2021-8-3':'2021-9-23']=df_turbidty['turbidty']['2021-8-3':'2021-9-23']+1.5
    df_turbidty['turbidty']['2021-9-26':'2021-10-17 15:00']=df_turbidty['turbidty']['2021-9-26':'2021-10-17 15:00']+2
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],144,0)       

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,3)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'隘寮_霧大_test.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371244/feeds.json?',
                       '隘寮_霧大', '2021-9-30', '2021-10-30', '2', 30.1, 0.015, 6.5, 3.9, 0.95, (-3529), 5138)

# 南豐橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C0L010廬山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel',filter_times=20,filter_value=0.08)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------


    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],24,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+1
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-8-3':'2021-8-30']=df_turbidty['turbidty']['2021-8-3':'2021-8-30']

    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,10)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'南豐橋_1030-1205.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'南豐橋_原始數據_0401-0721.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457351/feeds.json?',
                       '南豐橋', '2022-4-1 00:00', '2022-7-25 00:00', '2', 12.3, 0.045, 2, 5.63, 1, (-3529), 5138)

# 同德橋

In [ ]:

def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C0L010廬山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]=df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"].rolling(5,min_periods=1,center=True).max()

    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4


    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'同德橋_1030-1205.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'同德橋_原始數據_0401-0721.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457352/feeds.json?', '同德橋', '2022-4-1 00:00', '2022-7-25 00:00', '2', 12.3, 0.06, 2, 5.63, 1, (-3529), 5138)

# 夢谷一號橋

In [ ]:

def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\C0L010廬山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty',1)
 
    #做STL分析---------
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]=df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"].rolling(5,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty'][:'2022-5-23']=df_turbidty['turbidty'][:'2022-5-23']+1.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    # df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    # df_turbidty['turbidty']['2021-10-29':'2021-12-30']=6-df_turbidty['turbidty']['2021-10-29':'2021-12-30']
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'夢谷一號橋_1030-1205.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'夢谷一號橋_原始數據_0401-0721.xlsx')
    

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457353/feeds.json?', '夢谷一號橋', '2022-4-1 00:00', '2022-7-25 00:00', '2', 12.3, 0.06, 2, 5.63, 1, (-3529), 5138)

# 巴莎娜下游左岸護岸

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',4.8,4.06,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,2.5,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]=df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"].rolling(5,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']-0.2
    # df_turbidty['turbidty']['2021-9-7':'2021-9-9']=df_turbidty['turbidty']['2021-9-7':'2021-9-9']+0.5
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    df_turbidty['turbidty'] = STL_ana( df_turbidty['turbidty'],48,0) 
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
    Qs_sr_waterlevel=Qs_sr_waterlevel.fillna(method='ffill')
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'巴莎娜下游左岸護岸_1030-1205.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'巴莎娜下游左岸護岸_原始數據_0401-0721.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457378/feeds.json?api_key=0SCL3ZS6L2VYUMMP', '巴莎娜下游左岸護岸',
                       '2022-4-1 00:00', '2022-7-25 00:00', '1', 52, 0.05, 2, 4.8, 1, (-3529), 5138)

# 巴莎娜下游右岸支流

In [ ]:

def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',1,filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------


    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    df_waterlevel["waterlevel"]['2022-5-18 00:00':'2022-5-25']=df_waterlevel["waterlevel"]['2022-5-18 00:00':'2022-5-25'].rolling(60,min_periods=1,center=True).min().rolling(30,min_periods=1,center=True).min()
    df_waterlevel["waterlevel"]['2022-5-21 00:00':'2022-5-22']=df_waterlevel["waterlevel"]['2022-5-21 00:00':'2022-5-22'].rolling(60,min_periods=1,center=True).min().rolling(30,min_periods=1,center=True).min()

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'巴莎娜下游右岸支流_1030-1205.xlsx')    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)

    correction.to_excel(r'巴莎娜下游右岸支流_原始數據_0401-0721.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457376/feeds.json?api_key=FYXSI6WYJDCJ23EV', 
                       '巴莎娜下游右岸支流', '2022-4-1 00:00', '2022-7-25 00:00', '1', 5, 0.045, 2, 2.83, 1, (-3529), 5138)

# 巴莎娜上游

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------


    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   

    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'巴莎娜下游右岸支流_1030-1205.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457379/feeds.json?api_key=FYXSI6WYJDCJ23EV', 
                       '巴莎娜上游', '2022-5-1 00:00', '2022-7-25 00:00', '1', 5, 0.045, 2, 2.83, 1, (-3529), 5138)

# 來吉行諄橋

In [127]:

def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',filter_times=1,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------


    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   

    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+1.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'巴莎娜下游右岸支流_1030-1205.xlsx')   
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)

    correction.to_excel(r'來吉行諄橋_原始數據_0610-0721.xlsx')

In [128]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725548/feeds.json?api_key=FYXSI6WYJDCJ23EV', 
                       '來吉行諄橋', '2022-6-10 00:00', '2022-7-25 00:00', '1', 20, 0.045, 2, 4.38, 0.98, (-3529), 5138)

2022-6-10 00:00 2022-6-10 00:00
data is exist!
before filter point =  6110
after filter point =  6135
waterlevel    4.38
dtype: float64


turbidty    0.25
dtype: float64


===================summary=========================
累積雨量= 219.5 (mm)
懸移質輸砂量= 14142 (m3)
===================summary=========================


# 新美大橋

In [129]:

def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\81M660阿里山國小20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',filter_times=1,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------


    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   

    # df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+1.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'巴莎娜下游右岸支流_1030-1205.xlsx')   
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)

    correction.to_excel(r'新山美大橋_原始數據_0610-0721.xlsx')

In [132]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725549/feeds.json?api_key=FYXSI6WYJDCJ23EV', 
                       '新山美大橋', '2022-6-10 00:00', '2022-7-25 00:00', '1', 20, 0.045, 2, 4.38, 0.98, (-3529), 5838)

2022-6-10 00:00 2022-6-10 00:00
data is exist!
before filter point =  0
after filter point =  0
waterlevel   NaN
dtype: float64


turbidty    0.54
dtype: float64


===================summary=========================
累積雨量= 0.0 (mm)
懸移質輸砂量= 0 (m3)
===================summary=========================


# 廍仔坑溪下游政興橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\龍安.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,9.5,'waterlevel',3)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]=df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"].rolling(5,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(30,min_periods=1,center=True).max()
    

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']-0.5
    df_turbidty['turbidty']['2021-8-9':'2021-8-14']=4.6
    df_turbidty['turbidty']['2021-7-18':'2021-8-9']=df_turbidty['turbidty']['2021-7-18':'2021-8-9']-1
    
    df_turbidty['turbidty']['2021-8-9':'2021-8-30']=df_turbidty['turbidty']['2021-8-9':'2021-8-30']-0.7

    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'廍仔坑溪下游政興橋坑_原始數據_0516-0713.xlsx')
    res.to_excel(r'廍仔坑溪下游政興橋_20220516-20220713.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371242/feeds.json?api_key=6NR09BWCA5EX095V',
                       '廍仔坑溪下游政興橋', '2022-5-1 00:00', '2022-7-30 00:00', '1', 17.2, 0.035, 6.5, 9.9, 1, (-3109), 4618.9)

# 廍仔坑溪上游版橋右岸護岸

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\龍安.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,3,'waterlevel',5)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel["waterlevel"] = df_waterlevel["waterlevel"].rolling(600,min_periods=1,center=True).mean()

    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).min()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    df_waterlevel['2021-6-30':'2021-7-22']["waterlevel"]= df_waterlevel['2021-6-30':'2021-7-22']["waterlevel"]+0.1
    df_waterlevel['2021-7-9':'2021-7-22']["waterlevel"]= df_waterlevel['2021-7-9':'2021-7-22']["waterlevel"]+0.1

    df_waterlevel['2021-8-24':'2021-9-15']["waterlevel"]= df_waterlevel['2021-8-24':'2021-10-15']["waterlevel"]+0.05
    df_waterlevel['2021-8-31':'2021-9-15']["waterlevel"]= df_waterlevel['2021-8-31':'2021-10-15']["waterlevel"]+0.05
    df_waterlevel['2021-9-5':'2021-9-15']["waterlevel"]= df_waterlevel['2021-9-5':'2021-10-15']["waterlevel"]+0.05
    df_waterlevel['2021-9-8':'2021-9-12']["waterlevel"]= df_waterlevel['2021-9-8':'2021-9-12']["waterlevel"]+0.05
    df_waterlevel['2021-9-15':'2021-10-15']["waterlevel"]= df_waterlevel['2021-9-15':'2021-10-15']["waterlevel"]+0.07

    df_waterlevel['2021-9-16':'2021-9-25']["waterlevel"]= df_waterlevel['2021-9-16':'2021-9-25']["waterlevel"]+0.15
    df_waterlevel['2021-9-26':'2021-10-15']["waterlevel"]= df_waterlevel['2021-9-26':'2021-10-15']["waterlevel"]+0.1
    df_waterlevel['2021-9-21':'2021-9-25']["waterlevel"]= df_waterlevel['2021-9-21':'2021-9-25']["waterlevel"]+0.03

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = ((distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))) 

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,30)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty'].rolling(20,min_periods=1,center=True).max()
    df_turbidty['turbidty']['2021-8-8':'2021-10-30']=df_turbidty['turbidty']['2021-8-8':'2021-10-30']+1.3
    df_turbidty['turbidty']['2021-8-13':'2021-10-30']=df_turbidty['turbidty']['2021-8-13':'2021-10-30']+1
    df_turbidty['turbidty']['2021-6-25':'2021-7-31']=df_turbidty['turbidty']['2021-6-25':'2021-7-31']+2
    df_turbidty['turbidty']['2021-9-15':'2021-10-15']=df_turbidty['turbidty']['2021-9-15':'2021-10-15']+2
    df_turbidty['turbidty']['2021-6-26':'2021-7-22']=df_turbidty['turbidty']['2021-6-26':'2021-7-22']+2

    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'廍仔坑溪上游版橋右岸護岸_原始數據_0516-0713.xlsx')
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'廍仔坑溪上游版橋右岸護岸_20220516-20220713.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1371243/feeds.json?api_key=DSX6R7G4CB8GXHRU', '廍仔坑溪上游版橋右岸護岸',
                       '2022-5-17 00:00', '2022-7-17 00:00', '1', 26, 0.015, 6.5, 3.95, 1, (-4773), 7542.2)

# 廍仔坑溪上游調節壩

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\龍安.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',4.13,3.92,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------    

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],144,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   


    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,10)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'廍仔坑溪上游調節壩_原始數據_0517-0712.xlsx')
    res.to_excel(r'廍仔坑溪上游調節壩_20220517-20220712.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1713736/feeds.json?api_key=BIN53X93UZSKN5KC',
                       '廍仔坑溪上游調節壩', '2022-5-10 12:00', '2022-7-17', '2', 45, 0.015, 6.5, 3.81, 0.83, (-3109), 4618.2)

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\07.水砂資料處禮\健檢期末\阿里山國小.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,0,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]=df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"].rolling(5,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],12,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,3)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5
    #df_turbidty['turbidty']['2021-6-21':'2021-7-14']=df_turbidty['turbidty']['2021-6-21':'2021-7-14']+4
    #df_turbidty['turbidty']['2021-7-30':'2021-8-6']=df_turbidty['turbidty']['2021-7-30':'2021-8-6']+3.5
    #df_turbidty['turbidty']['2021-7-30':'2021-7-31']=df_turbidty['turbidty']['2021-7-30':'2021-7-31']+3.5
    #df_turbidty['turbidty']['2021-8-8':'2021-8-14']=df_turbidty['turbidty']['2021-8-8':'2021-8-14']+3.5
    #df_turbidty['turbidty']['2021-8-15':'2021-8-31']=df_turbidty['turbidty']['2021-8-15':'2021-8-31']+2.8
    #df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']=df_turbidty['turbidty']['2021-7-24 16:00:00':'2021-7-30 07:00:00']+1
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,Qs_sr_waterlevel,sr_field5,sr_field6)
    res.to_excel(r'巴莎娜上游防砂壩_1030-1205.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457379/feeds.json?api_key=GKDEZAA1JL17FQGT', '巴莎娜上游防砂壩', '2021-10-30', '2021-12-5', '1', 5, 0.045, 2, 2.83, 1, (-3529), 5138)

# 花蓮荖溪重光橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\花蓮20220101-20220613\瑞穗.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,5.3,'waterlevel',filter_value=0.15,filter_times=10)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
 
    #做STL分析---------
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).max()
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).mean()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],24,0)       
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,1)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5   
    #df_turbidty['turbidty']['2022-6-8':'2022-6-16'] = df_turbidty['turbidty']['2022-6-8':'2022-6-16']+2

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,df_waterlevel,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'花蓮荖溪重光橋-202200517-20220613.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'花蓮荖溪重光橋_原始數據_0517-0622.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725550/feeds.json?api_key=GKDEZAA1JL17FQGT',
                       '花蓮荖溪重光橋', '2022-5-17 00:00', '2022-6-22 00:00', '1', 17, 0.045, 2, 6.08, 1, (-3058), 4664)

# 花蓮 安夜西溪

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\花蓮20220101-20220613\瑞穗.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',4.4,4,'waterlevel',filter_times=10,filter_value=0.15)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    #df_waterlevel["waterlevel"]['2022-6-9':'2022-6-14']=df_waterlevel["waterlevel"]['2022-6-9':'2022-6-14'].apply(lambda x:4.35 if x <=4.25 else x)
 
    #做STL分析---------
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]['2022-5-28':'2022-6-8']=df_waterlevel["waterlevel"]['2022-5-28':'2022-6-8'].rolling(50,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6
    df_waterlevel["waterlevel"]['2022-5-26 00:00':'2022-5-28']=df_waterlevel["waterlevel"]['2022-5-26 00:00':'2022-5-28'].apply(lambda x:4.28 if x <=4.2 else x)
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"]+0.7
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(3,min_periods=1,center=True).max()

    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor)) 
    df_waterlevel["waterlevel"]['2022-6-1 07:00':'2022-6-1 19:00']=df_waterlevel["waterlevel"]['2022-6-1 07:00':'2022-6-1 19:00'].apply(lambda x:0.44 if x <=0.4 else x)
    df_waterlevel["waterlevel"]['2022-6-9':'2022-6-13']=df_waterlevel["waterlevel"]['2022-6-9':'2022-6-13'].rolling(20,min_periods=1,center=True).min()
    df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],24,0)       

    
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,50)
   
    #濁度========================================================================================#
   
    df_turbidty['turbidty']=df_turbidty['turbidty']+0.5   
    df_turbidty['turbidty']['2022-6-10':'2022-6-14 15:00']=df_turbidty['turbidty']['2022-6-10':'2022-6-14 15:00']-1 
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'花蓮安夜西溪-20220517-20220622.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'花蓮安夜西溪_原始數據_0517-0622.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725554/feeds.json?api_key=GKDEZAA1JL17FQGT',
                       '花蓮安夜西溪', '2022-5-1 00:00', '2022-6-22 00:00', '1', 10, 0.045, 2, 4.61, 0.84, (-3058), 4664)

# 花蓮 北坑一號橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\花蓮20220101-20220608\水蓮.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2.5,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,2,'turbidty')
 
    #做STL分析---------
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"]['2022-5-28':'2022-6-8']=df_waterlevel["waterlevel"]['2022-5-28':'2022-6-8'].rolling(50,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-3':'2021-8-5']["waterlevel"]+1
    #df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"]=df_waterlevel['2021-8-9':'2021-8-31']["waterlevel"].rolling(20,min_periods=1,center=True).max()
    #df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]=df_waterlevel['2021-8-8 15:00':'2021-8-30']["waterlevel"]+0.6

    #df_waterlevel["waterlevel"] = STL_ana( df_waterlevel["waterlevel"],48,0)
    
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))-1.3   
    mean=df_waterlevel["waterlevel"].mean()
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x >=0.3 else x)
    df_waterlevel["waterlevel"]['2022-5-18':'2022-5-21']=df_waterlevel["waterlevel"]['2022-5-18':'2022-5-21'].apply(lambda x:mean if x >=0.21 else x)
    df_waterlevel["waterlevel"]['2022-5-28':'2022-5-31']=df_waterlevel["waterlevel"]['2022-5-28':'2022-5-31'].apply(lambda x:mean if x >=0.21 else x)

    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#
   
    #df_turbidty['turbidty']=df_turbidty['turbidty']+0.5   
    #df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(10,min_periods=1,center=True).max().rolling(20,min_periods=1,center=True).mean()
    
    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,30)
     
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'北坑一號橋-20220518-20220622.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'北坑一號橋_原始數據_0517-0622.xlsx')

In [ ]:
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725556/feeds.json?api_key=GKDEZAA1JL17FQGT',
                       '北坑一號橋', '2022-5-18 00:00', '2022-6-22 00:00', '1', 7.5, 0.045, 2, 4.44, 0.84, (-3058), 4664)

# 後龍溪善東橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\花蓮20220101-20220608\水蓮.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',5,3,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    #mean=df_waterlevel["waterlevel"].mean()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+2.2 if x <=1.2 else x)
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x <=3.1 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # %matplotlib widget
    
    # df_waterlevel["waterlevel"].plot()
    #df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],5,3)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty'] = df_turbidty['turbidty'].rolling(50,min_periods=1,center=True).max()

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'善東橋_水砂盒0501-0622.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'善東橋_原始數據_0501-0721.xlsx')
    

In [ ]:
#善東橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725545/feeds.json?',
                       '善東橋', '2022-05-01 00:00', '2022-07-22 00:00','2', 12, 0.035, 5, 4.56, 0.86, (-3953), 6244.6)

# 南化木瓜坑

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\01O760關山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',5,2.5,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    #mean=df_waterlevel["waterlevel"].mean()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+2.2 if x <=1.2 else x)
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x <=3.1 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # %matplotlib widget
    
    # df_waterlevel["waterlevel"].plot()
    #df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],5,3)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty'] = df_turbidty['turbidty']+1#.rolling(50,min_periods=1,center=True).max()

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'南化木瓜坑橋護岸_水砂盒0528-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'南化木瓜坑橋護岸_原始數據_0528-0712.xlsx')
    

In [ ]:
#善東橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725606/feeds.json?',
                       '南化木瓜坑橋護岸', '2022-05-28 00:00', '2022-07-15 00:00','1', 15.5, 0.045, 1.1, 4.14, 0.86, (-3953), 6244.6)

# 南化平坑

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\01O760關山20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',5,2,'waterlevel',filter_times=10,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    #mean=df_waterlevel["waterlevel"].mean()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+2.2 if x <=1.2 else x)
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x <=3.1 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    #df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # %matplotlib widget
    
    # df_waterlevel["waterlevel"].plot()
    #df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],5,3)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty'] = df_turbidty['turbidty']+1.5#.rolling(50,min_periods=1,center=True).max()

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'南化平坑_水砂盒0528-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'南化平坑_原始數據_0528-0712.xlsx')
    

In [ ]:
#南化平坑
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725608/feeds.json?',
                       '南化平坑', '2022-05-27 00:00', '2022-07-30 00:00','1', 6, 0.045, 1.1, 3.6, 0.77, (-3953), 6244.6)

# 南化亞美坑

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"C:\Users\2235\Google 雲端硬碟 (n66051223@gs.ncku.edu.tw)\02.python_專案工作區\15.氣象局雨量計資料處理\石門&健檢雨量站資料20220101-20220715\01N1A大竹坑20220101-20220715.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',5,3.25,'waterlevel',filter_times=10,filter_value=0.08)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,1,'turbidty')
    
    #做STL分析---------
    #mean=df_waterlevel["waterlevel"].mean()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+2.2 if x <=1.2 else x)
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x <=3.1 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # %matplotlib widget
    
    # df_waterlevel["waterlevel"].plot()
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],17,11)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    # df_turbidty['turbidty'] = df_turbidty['turbidty']#.rolling(50,min_periods=1,center=True).max()
    df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],144,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'南化亞美坑_水砂盒0528-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'南化亞美坑_原始數據_0528-0712.xlsx')
    

In [ ]:
#亞美坑
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1725605/feeds.json?',
                       '亞美坑', '2022-05-28 00:00', '2022-07-22 00:00','1', 17.5, 0.025, 5, 2.8, 0.82, (-3953), 6244.6)

# 明德永興一號橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\獅潭.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,6.7,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    #mean=df_waterlevel["waterlevel"].mean()
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+2.2 if x <=1.2 else x)
    #df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:mean if x <=3.1 else x)
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).max()
    df_waterlevel['2022-6-17':'2022-6-24 13:00']=filter_by_differ(df_waterlevel['2022-6-17':'2022-6-24 13:00'],times=40,value=0.1)
    df_waterlevel['2022-6-25':'2022-6-29 17:00']=filter_by_differ(df_waterlevel['2022-6-25':'2022-6-29 17:00'],times=40,value=0.1)
    df_waterlevel['2022-6-30':'2022-7-10']=filter_by_differ(df_waterlevel['2022-6-30':'2022-7-10'],times=40,value=0.15)
    # df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # %matplotlib widget
    
    # df_waterlevel["waterlevel"].plot()
    df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    
    
    # std=2*(df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).std())
    # mean= df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).mean()
    # upper=(mean+std)
    # print(mean)
    # print(upper)
    
    # lower=(mean-std)
    # import matplotlib.pyplot as plt
    # plt.plot( df_waterlevel["waterlevel"])
    # plt.plot( lower)
    # plt.plot( upper)
    # # plt.fill_between(std.index,lower,upper,color='red')
    # plt.show()
    
    
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'] = df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'].rolling(100,min_periods=1,center=True).max()
    df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'明德永興一號橋_水砂盒0617-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'明德永興一號橋_原始數據_0617-0712.xlsx')
    return df_waterlevel["waterlevel"]

In [ ]:
#明德永興一號橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
aa=fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1766274/feeds.json?',
                       '明德永興一號橋', '2022-06-17 00:00', '2022-07-12 00:00','1', 18.2, 0.025, 5, 9, 1, (-3953), 6244.6)

--- 

# 明德永善橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\獅潭.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,1.5,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------

    # df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],144,0)
    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    df_waterlevel["waterlevel"].plot()
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'] = df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'].rolling(100,min_periods=1,center=True).max()
    # df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'明德永善橋_水砂盒0617-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'明德永善橋_原始數據_0617-0712.xlsx')

In [ ]:
#明德永善橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1766276/feeds.json?',
                       '明德永善橋', '2022-06-17 00:00', '2022-07-22 00:00','1', 10, 0.025, 5, 3.67, 0.87, (-3953), 6244.6)

# 明德大東勢一橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\獅潭.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',4.2,2.3,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0.5,'turbidty')
    
    #做STL分析---------

    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(5,min_periods=1,center=True).min()
    df_waterlevel['2022-6-17':'2022-6-24 13:00']=filter_by_differ(df_waterlevel['2022-6-17':'2022-6-24 13:00'],times=40,value=0.05)
    df_waterlevel['2022-6-25':'2022-6-28 17:00']=filter_by_differ(df_waterlevel['2022-6-25':'2022-6-28 17:00'],times=40,value=0.05)

    df_waterlevel['2022-6-30':'2022-7-11']=filter_by_differ(df_waterlevel['2022-6-30':'2022-7-11'],times=40,value=0.05)
    # df_waterlevel=filter_by_differ(df_waterlevel,times=10,value=0.1)
    # df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],24,0)

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'] = df_turbidty['turbidty']['2022-6-17':'2022-6-24 13:00'].rolling(100,min_periods=1,center=True).max()
    df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'明德大東勢一橋_水砂盒0617-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'明德大東勢一橋_原始數據_0617-0712.xlsx')

In [ ]:
#明德永善橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1766278/feeds.json?',
                       '明德大東勢一橋', '2022-06-17 00:00', '2022-07-22 00:00','1', 10, 0.025, 5, 4.21, 0.8, (-3953), 6244.6)

# 抽藤坑

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\龍安.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,method ='ffill')
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,value =3.34)



    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel['2022-7-5':'2022-7-11']=filter_by_differ(df_waterlevel['2022-7-5':'2022-7-11'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.37,method ='ffill')
    # df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10']=df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10'].apply(lambda x:1.07 if x >=1.76 else x)
    df_waterlevel['2022-5-26':'2022-6-10']=filter_by_differ(df_waterlevel['2022-5-26':'2022-6-10'],times=40,value=0.1)
    df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12'].rolling(10,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8']=df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8'].apply(lambda x:1.21 if x <=1.15 else x)
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:0.8 if x <=0.7 else x)

    # df_waterlevel=filter_by_differ(df_waterlevel,times=10,value=0.1)
    df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],48,0)

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty']= df_turbidty['turbidty']+4
    # df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'抽藤坑_水砂盒0520-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'抽藤坑_原始數據_0520-0712.xlsx')

In [ ]:
#抽藤坑
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457355/feeds.json?',
                       '抽藤坑', '2022-05-20 00:00', '2022-07-22 00:00','1', 32, 0.045, 1, 4.44,1.05, (-3953), 6244.6)

# 北田大橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\中竹林.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',1,filter_times=2,filter_value=0.1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,method ='ffill')
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,value =3.34)



    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel['2022-7-5':'2022-7-11']=filter_by_differ(df_waterlevel['2022-7-5':'2022-7-11'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.37,method ='ffill')
    # df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10']=df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10'].apply(lambda x:1.07 if x >=1.76 else x)
    # df_waterlevel['2022-5-26':'2022-6-10']=filter_by_differ(df_waterlevel['2022-5-26':'2022-6-10'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12'].rolling(10,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8']=df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8'].apply(lambda x:1.21 if x <=1.15 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:0.8 if x <=0.7 else x)

    # df_waterlevel=filter_by_differ(df_waterlevel,times=10,value=0.1)
    # df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],48,0)

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    # df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'北田大橋˙_水砂盒0520-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'北田大橋˙_原始數據_0520-0712.xlsx')

In [ ]:
#明德永善橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1713735/feeds.json?',
                       '北田大橋˙', '2022-05-20 00:00', '2022-07-22 00:00','1', 34.5, 0.045, 1, 4.62,1, (-3953), 6244.6)

# 頭汴坑龍寶橋上游護岸

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\中竹林.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,2,'waterlevel',1)
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.85,value=5.59)
    df_waterlevel["waterlevel"][:'2022-6-24']=df_waterlevel["waterlevel"][:'2022-6-24'].replace(to_replace=3.85,method='ffill')
    df_waterlevel["waterlevel"]['2022-6-24':]=df_waterlevel["waterlevel"]['2022-6-24':].replace(to_replace=3.85,value=np.nan)


    #做STL分析---------
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,method ='ffill')
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,value =3.34)
    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:x+1.94 if x <=3.9 else x)
    df_waterlevel[:'2022-6-24']=filter_by_differ(df_waterlevel[:'2022-6-24'],times=30,value=0.1)
    df_waterlevel['2022-6-24':'2022-6-26']=filter_by_differ(df_waterlevel['2022-6-24':'2022-6-26'],times=5,value=0.15)
    df_waterlevel['2022-6-27':]=filter_by_differ(df_waterlevel['2022-6-27':],times=1,value=0.15,interpolate=False)

    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    df_waterlevel["waterlevel"]['2022-6-8':'2022-6-11']=df_waterlevel["waterlevel"]['2022-6-8':'2022-6-11'].apply(lambda x:2.11 if x >2.11 else x)
    df_waterlevel["waterlevel"]['2022-7-1':'2022-7-6']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-6'].apply(lambda x:np.nan if x >3 else x)
    df_waterlevel["waterlevel"]['2022-7-1':'2022-7-7']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-7'].apply(lambda x:np.nan if x <2.4 else x)

    df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].interpolate()
    df_waterlevel['2022-6-27':]=filter_by_differ(df_waterlevel['2022-6-27':],times=1,value=0.08)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.37,method ='ffill')
    # df_waterlevel['2022-5-26':'2022-6-10']=filter_by_differ(df_waterlevel['2022-5-26':'2022-6-10'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12'].rolling(10,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8']=df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8'].apply(lambda x:1.21 if x <=1.15 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:0.8 if x <=0.7 else x)

    # df_waterlevel=filter_by_differ(df_waterlevel,times=10,value=0.1)
    df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],6,0)

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#

    df_turbidty['turbidty']['2022-5-25':] =df_turbidty['turbidty']['2022-5-25':]+3

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    res.to_excel(r'龍寶橋上游護岸_水砂盒0520-0722.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'龍寶橋上游護岸_原始數據_0520-0722.xlsx')

In [ ]:
#明德永善橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1713734/feeds.json?',
                       '龍寶橋上游護岸', '2022-05-20 00:00', '2022-07-22 00:00','1', 42.5, 0.05, 1, 7.86,1, (-3953), 6244.6)

# 美優橋

In [ ]:
def fix_sendbox_thinkspeak(device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction_factor, Qs_a, Qs_b):
    df,df_rain=getting_data(device_post_api,utc_time_start,utc_time_end,device_name,
                            r"O:\防災中心-Project\TD22003_111年中分水砂觀測及構造物巡查\01.各階段報告\01.報告\02.期中報告\守\中竹林.csv")
    df_waterlevel,sr_field5=original_data_process(df,'field5',10,3.84,'waterlevel')
    df_turbidty,sr_field6=original_data_process(df,'field6',5,0,'turbidty')
    
    #做STL分析---------
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(20,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,method ='ffill')
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.19,value =3.34)



    df_waterlevel["waterlevel"] = (distance) - (((df_waterlevel["waterlevel"])) * (correction_factor))   
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].rolling(10,min_periods=1,center=True).min()
    # df_waterlevel['2022-7-5':'2022-7-11']=filter_by_differ(df_waterlevel['2022-7-5':'2022-7-11'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)

    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].replace(to_replace=3.37,method ='ffill')
    # df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10']=df_waterlevel["waterlevel"]['2022-5-26':'2022-6-10'].apply(lambda x:1.07 if x >=1.76 else x)
    # df_waterlevel['2022-5-26':'2022-6-10']=filter_by_differ(df_waterlevel['2022-5-26':'2022-6-10'],times=40,value=0.1)
    # df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12']=df_waterlevel["waterlevel"]['2022-7-1':'2022-7-12'].rolling(10,min_periods=1,center=True).max()
    # df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8']=df_waterlevel["waterlevel"]['2022-7-3':'2022-7-8'].apply(lambda x:1.21 if x <=1.15 else x)
    # df_waterlevel["waterlevel"]=df_waterlevel["waterlevel"].apply(lambda x:0.8 if x <=0.7 else x)

    # df_waterlevel=filter_by_differ(df_waterlevel,times=10,value=0.1)
    # df_waterlevel["waterlevel"] = STL_ana(df_waterlevel["waterlevel"],48,0)

    # df_waterlevel["waterlevel"]=savgol_filter(df_waterlevel["waterlevel"],7,3)
    Q_sr_waterlevel=Q_calculate(df_waterlevel,width,slope,man,mode,10)
   
    #濁度========================================================================================#
    df_turbidty['turbidty']=df_turbidty['turbidty']+0.3
    df_turbidty['turbidty'] = STL_ana(df_turbidty['turbidty'],48,0)

    Qs_sr_waterlevel,sr_turbidty=Qs_calculate(df_turbidty,Qs_a,Qs_b,Q_sr_waterlevel,50)
 
    #---draw-----------------------
    draw(df_waterlevel,sr_field5,sr_field6,Q_sr_waterlevel,Qs_sr_waterlevel,df_rain,sr_turbidty,device_name, utc_time_start, utc_time_end)
    #---save-----------------------
    res=save_data(utc_time_start,utc_time_end,df_rain,Q_sr_waterlevel,
                  Qs_sr_waterlevel,sr_field5,sr_field6,df_waterlevel["waterlevel"])
    # res.to_excel(r'北田大橋˙_水砂盒0520-0712.xlsx')
    correction=correction_data_save(Q_sr_waterlevel=Q_sr_waterlevel,Qs_sr_waterlevel=Qs_sr_waterlevel,waterlevel=df_waterlevel["waterlevel"],sr_turbidty=sr_turbidty)
    correction.to_excel(r'美優橋_原始數據_0520-0712.xlsx')

In [ ]:
#美優橋
#device_post_api, device_name, utc_time_start, utc_time_end, mode, width, man, slope, distance, correction
fix_sendbox_thinkspeak('https://api.thingspeak.com/channels/1457380/feeds.json?',
                       '美優橋', '2022-05-20 00:00', '2022-07-22 00:00','1', 42.5, 0.05, 1, 7.86,1, (-3953), 6244.6)

In [ ]:
sb.raw_data['field5']
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
#plot_pacf(sb.raw_data['field5'].astype(float).diff().dropna())
from statsmodels.tsa.stattools import adfuller
result=adfuller(sb.raw_data['field5'].astype(float).dropna())
from statsmodels.tsa.arima.model import ARIMA 
arima_model=ARIMA(sb.raw_data['field5'].astype(float),order=(2,2,1))
model=arima_model.fit()
print(model.summary())
sb.raw_data['field5']
model.plot_diagnostics()

# plot.show()

In [ ]:
pd.to_datetime(sb.raw_data.time)

In [ ]:
sb.raw_data=sb.raw_data.set_index('time',drop=True)
sb.raw_data


In [ ]:
import matplotlib.pyplot as plt
data=5.49-sb.raw_data['field5'].astype(float)
#data=data.dropna()
plt.plot(data)

In [ ]:
sb.raw_data
data_diff_1=data.diff()
data=data[data_diff_1<=0.1][data_diff_1>=(-0.1)]
data_diff_2=data.diff(2)
data=data[data_diff_2<=0.3][data_diff_2>=(-0.3)]
plt.plot(data)
data.max()

In [ ]:
aa

In [ ]:
import matplotlib.pyplot as plt

print(std.index)
plt.plot(aa)


plt.fill_between(std.index,lower,upper,color='red')
plt.show()

In [ ]:
df=pd.DataFrame({'aa':aa,"upper":upper,'lower':lower})
# condition1=df['aa']>df['upper']
# condition2=df['aa']<df['lower']
# df.drop(df.loc[condition1|condition2].index,axis=0)
df['aa'].plot()
df['aa']


In [ ]:
bb=filter_by_std(aa,-4.1)
bb.plot()

In [ ]:
bb=filter_by_std(aa,-4.1)
bb.plot()

In [ ]:

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
data=data.reset_index(drop=True)
#result=adfuller(data.dropna())
#plot_pacf(data)
from statsmodels.tsa.arima.model import ARIMA 
size=int(len(data)/2)
train=data[:size]
test=data[size:]
arima_model=ARIMA(train,order=(9,1,3))
model=arima_model.fit()
print(model.summary())
model.plot_diagnostics()


In [ ]:
data.reset_index(drop=True,inplace=True)
from statsmodels.graphics.tsaplots import plot_predict
plot_predict(test)

In [ ]:
start=len(train)
end=len(data)-1
pre=model.predict(start=start,end=end,return_conf_int=True)
#pre=model.forecast(len(test),return_conf_int=True)

In [ ]:
pre.plot()
train.plot()

In [ ]:
pre

In [ ]:
train[-100:].plot()
pre[:20].plot()

In [ ]:
fr=model.forecast(10000)

In [ ]:
sb.raw_data
# data_diff_1=data.diff()
# data=data[data_diff_1<=0.1][data_diff_1>=(-0.1)]
# data_diff_2=data.diff(2)
# data=data[data_diff_2<=0.3][data_diff_2>=(-0.3)]
# plt.plot(data)
# data.max()

In [ ]:
df=sb.raw_data
field='field5'
high=10
low=2
name='waterlevel'
time=[]
value=[]
for i in range(len(df)):
    if float(df[field][i]) >low and float(df[field][i]) < high and float(df[field][i]) !=0.001:       
        time.append(df['time'][i])
        value.append(float(df[field][i])) 

data={'time':time,name:value}
data1 = pd.DataFrame(data)
data1['time']=pd.to_datetime(data1['time'])
data1 = data1.set_index("time", drop=True) 
data1_differ=filter_by_differ(data1,0.1,15)
data1_differ.plot()
data1.plot()

In [ ]:
sb.filter_data.plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
resample_data=sb.filter_data.resample('10T').mean().interpolate(limit = 1)
data=pd.Series(resample_data['waterlevel'].dropna())

data.plot()
#data=data.reset_index(drop=True)
result=adfuller(data)
plot_pacf(data)
from statsmodels.tsa.arima.model import ARIMA 
# size=int(len(data)/2)
# train=data[:size]
# test=data[size:]
arima_model=ARIMA(data,order=(7,3,4))
model=arima_model.fit()
print(model.summary())
model.plot_diagnostics()


In [ ]:
# start=len(train)
#end=len(data)-1
pre=model.predict(start='2022-05-02',end='2022-06-5',return_conf_int=True)
pre.plot()
#resample_datap[2:].plot()

In [ ]:
#pre.resample('10T').mean().interpolate().plot()
pre.plot()
resample_data[2:]['waterlevel'].plot()

In [ ]:
resample_data[resample_data.isna()]

In [ ]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
from numpy import ma


In [ ]:
resample_data[2:]['waterlevel'].dropna().values

In [ ]:

# enable or disable missing observations
use_mask = 1

# reading data (quick and dirty)

Time=resample_data[2:]['waterlevel'].dropna().index
X=resample_data[2:]['waterlevel'].dropna().values


if (use_mask):
    X = ma.asarray(X)
    X[1300:2500] = ma.masked

# Filter Configuration

# time step
dt = 0.10
print(str(dt))

# transition_matrix  
F = [[1,  dt,   0.5*dt*dt], 
     [0,   1,          dt],
     [0,   0,           1]]  

# observation_matrix   
H = [1, 0, 0]

# transition_covariance 
Q = [[   1,     0,     0], 
     [   0,  1e-4,     0],
     [   0,     0,  1e-6]] 

# observation_covariance 
R = [0.04] # max error = 0.6m

# initial_state_mean
X0 = [0,
      0,
      0]

# initial_state_covariance
P0 = [[ 10,    0,   0], 
      [  0,    1,   0],
      [  0,    0,   1]]

n_timesteps = len(Time)
n_dim_state = 3

filtered_state_means = np.zeros((n_timesteps, n_dim_state))
filtered_state_covariances = np.zeros((n_timesteps, n_dim_state, n_dim_state))

# Kalman-Filter initialization
kf = KalmanFilter(transition_matrices = F, 
                  observation_matrices = H, 
                  transition_covariance = Q, 
                  observation_covariance = R, 
                  initial_state_mean = X0, 
                  initial_state_covariance = P0)


# iterative estimation for each new measurement
for t in range(n_timesteps):
    if t == 0:
        filtered_state_means[t] = X0
        filtered_state_covariances[t] = P0
    else:
        filtered_state_means[t], filtered_state_covariances[t] = (
        kf.filter_update(
            filtered_state_means[t-1],
            filtered_state_covariances[t-1],
            observation = X[t])
        )

position_sigma = np.sqrt(filtered_state_covariances[:, 0, 0]);        

# plot of the resulted trajectory        
plt.plot(Time, filtered_state_means[:, 0], "g-", label="Filtered position", markersize=1)
plt.plot(Time, filtered_state_means[:, 0] + position_sigma, "r--", label="+ sigma", markersize=1)
plt.plot(Time, filtered_state_means[:, 0] - position_sigma, "r--", label="- sigma", markersize=1)
plt.grid()
plt.legend(loc="upper left")
plt.xlabel("Time (s)")
plt.ylabel("Position (m)")
plt.show()      

In [ ]:
data=resample_data.resample('H').mean()

# KNN moel

In [ ]:
def knn_process(df):
    from fancyimpute import KNN
    #先把資料時間校正回10分鐘一筆，有缺的用移漏值代替
    resample_data=sb.filter_data.resample('10T').mean().interpolate(limit = 1)

    #資料取出來
    data=resample_data


    #knn訓練資料前處理

    kn_value=48 #決定幾個為一組訓練
    raw_len=len(data['waterlevel'].values)  #計算原資料長度
    dim_n=int(raw_len/kn_value)  #計算要切成幾層，然後取整數
    after_len=dim_n*kn_value #計算總資料筆數

    #把時間序列轉換成矩陣
    kn_matrix=data['waterlevel'].values[:after_len].reshape((dim_n,kn_value))

    #KNN訓練
    knn_impute = KNN(k = 5,).fit_transform(kn_matrix)

    #把補遺後資料flatten
    aaa=knn_impute.reshape(1,after_len)
    ww=pd.Series(aaa[0])

    #跟原本資料合併
    knn_data=resample_data.iloc[:after_len]
    ww.index=knn_data.index
    knn_data['filter_data']=ww
    #knn_data.plot()
    knn_data=knn_data[knn_data['filter_data']!=0]

    # #畫圖
    # linear=resample_data['waterlevel'].interpolate()

    # trace_raw = go.Scatter(
    #     x= knn_data.index,
    #     y= knn_data['waterlevel'],
    #     name='raw',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines+markers',
    #     line=dict(color='red', width=1)
    # )


    # trace_field5 = go.Scatter(
    #     x= linear.index,
    #     y= linear,
    #     name='linear',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines',
    #     line=dict(color='blue', width=1)
    # )

    # trace_field6 = go.Scatter(
    #     x= knn_data.index,
    #     y= knn_data['filter_data'],
    #     name='knn_plot',
    #     xaxis='x', 
    #     yaxis='y',
    #     mode='lines',
    #     line=dict(color='orange', width=1)
    # )
    # data = [trace_field5, trace_field6,trace_raw]
    # layout = go.Layout({"template": 'ggplot2',
    #                     "title":{"text": '{0}'.format('device_name'), "x": 0.5}, 
    #                     "xaxis":{"title": {"text": "時間"}, "tickformat": '', 'domain':[0, 0.8]},
    #                     "yaxis":{"title": {"text": "雨量"}},
                        
    #                 "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
    #                         })

    # fig = go.Figure(data=data, layout=layout)
    # fig.show()
    # knn_data.resample('H').mean()['filter_data'].plot()
        

return knn_data


In [ ]:
knn_data['filter_data'].rename('ss')


In [ ]:
knn_data.dropna(subset=['filter_data'])

In [ ]:
pd.Series(data['waterlevel'].values)

In [ ]:
sb.filter_data

In [ ]:
knn_data

# sg model

In [ ]:
sb.raw_data.field5.astype(float).plot()
data=sb.raw_data.field5.astype(float)

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
data_filter=savgol_filter(data,window_length=99,polyorder=3)
pd.Series(data_filter).plot()

In [ ]:

# %matplotlib widget
# import ipywidgets as widgets
# import matplotlib.pyplot as plt
# from matplotlib.widgets import Slider
# data_filter=savgol_filter(data,window_length=99,polyorder=3)
# fig=plt.figure()
# ax=fig.subplots()
# p=ax.plot(data,'-*')
# p,=ax.plot(data_filter,'r')
# plt.subplots_adjust(bottom=0.25)
# ax_slide=plt.axes([0.25,0.1,0.65,0.03])
# win_len=Slider(ax_slide,'Window length',valmin=5,valmax=99,valinit=99,valstep=2)
# def update(val):
#     current=int(win_len.val)
#     new=savgol_filter(data,current,7)
#     p.set_ydata(new)
#     fig.canvas.draw()
    
# win_len.on_changed(update)
# plt.show()

    
    


In [ ]:
def sgmodel(data):
    data_filter=savgol_filter(data,window_length=99,polyorder=3)
    return data_filter

In [ ]:
df=pd.read_excel(r'O:\防災中心-Project\TD22011_健檢十\14.水保局監測資料上傳\南化平坑_原始數據_0528-0705_光感式濁度計.xlsx')
from datetime import datetime, timedelta


df['time']=df['time'].apply(lambda x:datetime.strftime(x,'%Y%m%d%H%M%S'))
df['time']
# df.to_excel('南化平坑_原始數據_0528-0705_光感式濁度計.xlsx')

In [ ]:
datetime.strftime(df['time'][0],'%Y%m%d%H%M%S')

In [ ]:
# df['time']=pd.to_datetime(df['time'])

In [ ]:

# datetime.strftime(df['time'],'%Y%m%d%H%M%S')



# datetime.strptime(df['time'][0], '%Y%m%d%H%M%S')

In [ ]:
import time # 引入time
timeString = "2020-09-09 19:00:00"
struct_time = time.strptime(timeString, "%Y-%m-%d %H:%M:%S") # 轉成時間元組
time_stamp = int(time.mktime(struct_time)) # 轉成時間戳
print(time_stamp)

In [ ]:
from datetime import datetime
cc=datetime.strptime("2018-01-31", "%Y-%m-%d")

In [ ]:
cc